# Validate Model on Pynq
In this last step we're going to check if we can get the same accuracy as what we observed in 1-train-model-with-brevitas.ipynb. In my case, the accuracy I observed was 0.68.

In [1]:
import numpy as np
from driver import io_shape_dict
from driver_base import FINNExampleOverlay

In [2]:
bsize = 100 # Full test set
platform = "zynq-iodma" # My device

# Subject 3's data, I already quantized this in the same way that's done in models/quant_deep4.py
test_data = np.load('./data/subject3_test_X.npy').astype('int8')
labels = np.load('./data/subject3_test_Y.npy')

# My example bitfile, feel free to use your own and check it's accuracy
bitfile = "./bitfile/finn-accel.bit"
# bitfile = "./your-bitfile/finn-accel.bit"

In [3]:
driver = FINNExampleOverlay(
    bitfile_name=bitfile,
    platform=platform,
    io_shape_dict=io_shape_dict,
    batch_size=bsize,
    runtime_weight_dir="runtime_weights/",
)

In [4]:
# Create an empty buffer for receiving data from the device
obuf_packed = np.empty_like(driver.obuf_packed_device[0])

In [5]:
%%time
ibuf_folded = driver.fold_input(test_data[0:bsize])
ibuf_packed = driver.pack_input(ibuf_folded)
driver.copy_input_data_to_device(ibuf_packed)
driver.execute_on_buffers()
driver.copy_output_data_from_device(obuf_packed)
obuf_folded = driver.unpack_output(obuf_packed)
obuf_normal = driver.unfold_output(obuf_folded)

CPU times: user 575 ms, sys: 39.6 ms, total: 614 ms
Wall time: 612 ms


In [6]:
pred_classes = np.argmax(obuf_normal, axis=-1).reshape(-1)

In [7]:
# Compute accuracy
accuracy = np.mean(pred_classes == labels)
print("Accuracy:", accuracy)

Accuracy: 0.68
